## POS tagging using modified Viterbi

### Data Preparation

In [1]:
#Importing libraries
import nltk
import nltk, re, pprint
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pprint, time
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize

In [2]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [3]:
# Checking the data by 10 examples
print(nltk_data[:10])

[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')], [('Rudolph', 'NOUN'), ('Agnew', 'NOUN'), (',', '.'), ('55', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), ('and', 'CONJ'), ('former', 'ADJ'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Consolidated', 'NOUN'), ('Gold', 'NOUN'), ('Fields', 'NOUN'), ('PLC', 'NOUN'), (',', '.'), ('was', 'VERB'), ('named', 'VERB'), ('*-1', 'X'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('of', 'ADP'), ('this', 'DET'), ('British', 'ADJ'), ('industrial', 'ADJ'), ('

In [4]:
# Splitting into train and test
random.seed(1234)
train_set, test_set = train_test_split(nltk_data,train_size = 0.95, test_size=0.05,random_state = 101)

print(len(train_set))
print(len(test_set))
print(train_set[:40])

3718
196
[[('Reliance', 'NOUN'), ('confirmed', 'VERB'), ('the', 'DET'), ('filing', 'NOUN'), ('but', 'CONJ'), ('would', 'VERB'), ("n't", 'ADV'), ('elaborate', 'VERB'), ('.', '.')], [('*', 'X'), ('Encouraging', 'VERB'), ('long-term', 'ADJ'), ('investing', 'NOUN'), ('.', '.')], [('Because', 'ADP'), ('of', 'ADP'), ('the', 'DET'), ('rulings', 'NOUN'), (',', '.'), ('the', 'DET'), ('Commerce', 'NOUN'), ('Department', 'NOUN'), ('will', 'VERB'), ('continue', 'VERB'), ('*-1', 'X'), ('to', 'PRT'), ('investigate', 'VERB'), ('complaints', 'NOUN'), ('*ICH*-2', 'X'), ('by', 'ADP'), ('U.S.', 'NOUN'), ('sweater', 'NOUN'), ('makers', 'NOUN'), ('that', 'ADP'), ('the', 'DET'), ('imports', 'NOUN'), ('are', 'VERB'), ('reaching', 'VERB'), ('the', 'DET'), ('U.S.', 'NOUN'), ('at', 'ADP'), ('unfairly', 'ADV'), ('low', 'ADJ'), ('prices', 'NOUN'), ('in', 'ADP'), ('violation', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('U.S.', 'NOUN'), ('anti-dumping', 'ADJ'), ('act', 'NOUN'), ('.', '.')], [('What', 'PRON'), ('she',

We have 3718 data sets in train dataset and 196 in test dataset

In [5]:
# Getting list of tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
len(train_tagged_words)

95547

In [6]:
# tokens 
tokens = [pair[0] for pair in train_tagged_words]
print(tokens[:10])

['Reliance', 'confirmed', 'the', 'filing', 'but', 'would', "n't", 'elaborate', '.', '*']


In [7]:
# vocabulary
V = set(tokens)
print(len(V))

12100


In [8]:
print(V)

{'tackle', '*-2', '13.625', 'execs', 'school-sponsored', 'predicting', 'trap', '2.44', '*-105', 'Douglas', 'entity', 'plummeted', 'apartment', 'Unicorp', 'decay', 'Schlemmer', 'self-regulatory', 'Signs', '225,000', 'PaineWebber', 'hit', 'robustly', 'tell', 'committed', 'suffer', 'claiming', '139', 'Signore', 'convert', 'diabetes', 'Dealers', 'scenic', 'north', 'DD', 'start', 'acquire', '6.50', 'everybody', 'farmers', 'scream', 'rear-seat', 'lotter', 'authors', '4.75', 'also', 'His', 'counterattack', 'wage', 'Greer', 'incentive', 'folks', 'roof', 'incomes', 'richer', 'Muzzling', '*T*-165', '1.916', 'romance', 'Unfortunately', 'computer-assisted', 'professor', 'started', '*T*-60', 'wholesaler', 'Pierre', 'portrayed', 'openly', 'peddling', 'subjecting', '17.95', '*-139', 'transplant', 'tight', 'psychiatric', '*T*-63', 'disproportionate', 'alternatively', 'seat', 'elusive', 'Waste', 'Garanti', 'suing', '7.80', 'Dana-Farber', 'inventories', 'aid', 'Thomas', 'closed', 'contributions', 'trigg

In [9]:
# number of Unique tags
T = set([pair[1] for pair in train_tagged_words])
len(T)

12

In [10]:
# Print all the tags used in file 
print(T)

{'ADP', 'PRT', 'PRON', 'NOUN', 'VERB', 'DET', 'ADJ', 'X', 'CONJ', 'NUM', '.', 'ADV'}


as mentioned in assignment there are total 12 tags 

### Build the vanilla Viterbi based POS tagger

### Emission Probabilities

In [11]:
# computing P(w/t) and storing in T x V matrix
t = len(T)
v = len(V)
w_given_t = np.zeros((t, v))

In [12]:
# compute word given tag: Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)

### Transition Probabilities

In [13]:
# compute tag given tag: tag2(t2) given tag1 (t1), i.e. Transition Probability

def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [14]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [15]:
tags_matrix

array([[1.68929752e-02, 1.38992839e-03, 7.00310096e-02, 3.20966542e-01,
        8.33956990e-03, 3.24708641e-01, 1.07024483e-01, 3.44274566e-02,
        9.62258084e-04, 6.22260235e-02, 3.90249118e-02, 1.40062012e-02],
       [2.00988464e-02, 1.64744642e-03, 1.77924223e-02, 2.47775942e-01,
        4.05271828e-01, 9.78583172e-02, 8.30313042e-02, 1.35090612e-02,
        2.30642501e-03, 5.66721596e-02, 4.38220762e-02, 1.02141676e-02],
       [2.29709037e-02, 1.30168451e-02, 7.65696773e-03, 2.10949466e-01,
        4.85451758e-01, 9.95405857e-03, 7.31240436e-02, 8.99693742e-02,
        5.35987737e-03, 6.50842255e-03, 4.09647785e-02, 3.40735056e-02],
       [1.76513597e-01, 4.33971919e-02, 4.60661016e-03, 2.63563901e-01,
        1.47667453e-01, 1.29423812e-02, 1.22477328e-02, 2.91751977e-02,
        4.26659845e-02, 9.54226404e-03, 2.40603983e-01, 1.70737058e-02],
       [9.20216888e-02, 3.06738969e-02, 3.57862115e-02, 1.10069714e-01,
        1.69248641e-01, 1.34391949e-01, 6.49883822e-02, 2.17

In [16]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))
tags_df

,ADP,PRT,PRON,NOUN,VERB,DET,ADJ,X,CONJ,NUM,.,ADV
ADP,0.016893,0.001390,0.070031,0.320967,0.008340,0.324709,0.107024,0.034427,0.000962,0.062226,0.039025,0.014006
PRT,0.020099,0.001647,0.017792,0.247776,0.405272,0.097858,0.083031,0.013509,0.002306,0.056672,0.043822,0.010214
PRON,0.022971,0.013017,0.007657,0.210949,0.485452,0.009954,0.073124,0.089969,0.005360,0.006508,0.040965,0.034074
NOUN,0.176514,0.043397,0.004607,0.263564,0.147667,0.012942,0.012248,0.029175,0.042666,0.009542,0.240604,0.017074
VERB,0.092022,0.030674,0.035786,0.110070,0.169249,0.134392,0.064988,0.217506,0.005577,0.022851,0.034934,0.081952
DET,0.009540,0.000242,0.003744,0.638087,0.039850,0.005676,0.204323,0.045405,0.000483,0.022220,0.017993,0.012438
ADJ,0.078267,0.010710,0.000330,0.699621,0.011699,0.004943,0.066403,0.021091,0.016971,0.021256,0.063931,0.004778
X,0.142584,0.185232,0.055538,0.062381,0.203851,0.054742,0.017187,0.076384,0.010662,0.002864,0.163590,0.024984
CONJ,0.052534,0.004649,0.058113,0.349140,0.156671,0.121339,0.118085,0.008833,0.000465,0.039981,0.034868,0.055323
NUM,0.036033,0.026504,0.001489,0.350208,0.018761,0.003276,0.034247,0.210542,0.013699,0.184932,0.117332,0.002978


In [17]:
tags_df.loc['.', :]

ADP     0.091342
PRT     0.002427
PRON    0.066349
NOUN    0.222242
VERB    0.089095
DET     0.173335
ADJ     0.043963
X       0.026971
CONJ    0.057538
NUM     0.081003
.       0.093320
ADV     0.052324
Name: ., dtype: float32

##  Viterbi Algorithm

In [18]:
# Vanila Viterbi Heuristic
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

#### Testing vanila veterbi 

In [19]:
# Running on entire test dataset would take more time so 
# Let's test our Viterbi algorithm on a few sample sentences of test dataset

random.seed(1234)

# choose random 5 sents
rndom = [random.randint(1,len(test_set)) for x in range(5)]

# list of sents
test_run = [test_set[i] for i in rndom]

# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]
print(test_run)

[[('The', 'DET'), ('Contra', 'NOUN'), ('military', 'ADJ'), ('command', 'NOUN'), (',', '.'), ('in', 'ADP'), ('a', 'DET'), ('statement', 'NOUN'), ('from', 'ADP'), ('Honduras', 'NOUN'), (',', '.'), ('said', 'VERB'), ('0', 'X'), ('Sandinista', 'NOUN'), ('troops', 'NOUN'), ('had', 'VERB'), ('launched', 'VERB'), ('a', 'DET'), ('major', 'ADJ'), ('offensive', 'NOUN'), ('against', 'ADP'), ('the', 'DET'), ('rebel', 'NOUN'), ('forces', 'NOUN'), ('.', '.')], [('*-1', 'X'), ('Bucking', 'VERB'), ('the', 'DET'), ('market', 'NOUN'), ('trend', 'NOUN'), (',', '.'), ('an', 'DET'), ('issue', 'NOUN'), ('of', 'ADP'), ('$', '.'), ('130', 'NUM'), ('million', 'NUM'), ('*U*', 'X'), ('general', 'ADJ'), ('obligation', 'NOUN'), ('distributable', 'ADJ'), ('state', 'NOUN'), ('aid', 'NOUN'), ('bonds', 'NOUN'), ('from', 'ADP'), ('Detroit', 'NOUN'), (',', '.'), ('Mich.', 'NOUN'), (',', '.'), ('apparently', 'ADV'), ('drew', 'VERB'), ('solid', 'ADJ'), ('investor', 'NOUN'), ('interest', 'NOUN'), ('.', '.')], [('Ralston', 

In [20]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start

In [21]:
print("Time taken in seconds: ", difference)
print(tagged_seq)


Time taken in seconds:  38.03717541694641
[('The', 'DET'), ('Contra', 'ADP'), ('military', 'ADJ'), ('command', 'VERB'), (',', '.'), ('in', 'ADP'), ('a', 'DET'), ('statement', 'NOUN'), ('from', 'ADP'), ('Honduras', 'ADP'), (',', '.'), ('said', 'VERB'), ('0', 'X'), ('Sandinista', 'ADP'), ('troops', 'NOUN'), ('had', 'VERB'), ('launched', 'VERB'), ('a', 'DET'), ('major', 'ADJ'), ('offensive', 'ADP'), ('against', 'ADP'), ('the', 'DET'), ('rebel', 'ADP'), ('forces', 'NOUN'), ('.', '.'), ('*-1', 'X'), ('Bucking', 'ADP'), ('the', 'DET'), ('market', 'NOUN'), ('trend', 'NOUN'), (',', '.'), ('an', 'DET'), ('issue', 'NOUN'), ('of', 'ADP'), ('$', '.'), ('130', 'NUM'), ('million', 'NUM'), ('*U*', 'X'), ('general', 'ADJ'), ('obligation', 'NOUN'), ('distributable', 'ADJ'), ('state', 'NOUN'), ('aid', 'NOUN'), ('bonds', 'NOUN'), ('from', 'ADP'), ('Detroit', 'NOUN'), (',', '.'), ('Mich.', 'NOUN'), (',', '.'), ('apparently', 'ADV'), ('drew', 'ADP'), ('solid', 'ADJ'), ('investor', 'NOUN'), ('interest', 'NO

In [22]:
# Vanilla Viterbi Algorithm Accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Vanilla Viterbi Algorithm Accuracy: ',accuracy*100)

Vanilla Viterbi Algorithm Accuracy:  89.38053097345133


In [23]:
# let's check the incorrect tags
incorrect_tagged_cases = [[test_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0]!=j[1]]

In [24]:
incorrect_tagged_cases

[[('The', 'DET'), (('Contra', 'ADP'), ('Contra', 'NOUN'))],
 [('military', 'ADJ'), (('command', 'VERB'), ('command', 'NOUN'))],
 [('from', 'ADP'), (('Honduras', 'ADP'), ('Honduras', 'NOUN'))],
 [('0', 'X'), (('Sandinista', 'ADP'), ('Sandinista', 'NOUN'))],
 [('major', 'ADJ'), (('offensive', 'ADP'), ('offensive', 'NOUN'))],
 [('the', 'DET'), (('rebel', 'ADP'), ('rebel', 'NOUN'))],
 [('*-1', 'X'), (('Bucking', 'ADP'), ('Bucking', 'VERB'))],
 [('apparently', 'ADV'), (('drew', 'ADP'), ('drew', 'VERB'))],
 [('its', 'PRON'), (('Eveready', 'ADP'), ('Eveready', 'NOUN'))],
 [('what', 'PRON'), (('*T*-252', 'ADP'), ('*T*-252', 'X'))],
 [('*-1', 'X'), (('complaining', 'ADP'), ('complaining', 'VERB'))],
 [('ended', 'VERB'), (('up', 'ADV'), ('up', 'PRT'))]]

### Solve the problem of unknown words

### Viterbi Modification-Technique 1

In [25]:
# using transition probability of tags when emission probability is zero (in case of unknown words)

def Viterbi_1(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        p_transition =[] # list for storing transition probabilities
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            p_transition.append(transition_p)
            
        pmax = max(p)
        state_max = T[p.index(pmax)] 
        
      
        # if probability is zero (unknown word) then use transition probability
        if(pmax==0):
            pmax = max(p_transition)
            state_max = T[p_transition.index(pmax)]
                           
        else:
            state_max = T[p.index(pmax)] 
        
        state.append(state_max)
    return list(zip(words, state))

In [26]:

# tagging the test sentences
start = time.time()
tagged_seq = Viterbi_1(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)


Time taken in seconds:  34.37096619606018


In [27]:
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Modified Viterbi_1 Accuracy: ',accuracy*100)

Modified Viterbi_1 Accuracy:  94.69026548672566


In [28]:
# let's check the incorrect tags
incorrect_tagged_cases = [[test_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0]!=j[1]]

In [29]:
incorrect_tagged_cases

[[('military', 'ADJ'), (('command', 'VERB'), ('command', 'NOUN'))],
 [('from', 'ADP'), (('Honduras', 'DET'), ('Honduras', 'NOUN'))],
 [('0', 'X'), (('Sandinista', 'VERB'), ('Sandinista', 'NOUN'))],
 [('its', 'PRON'), (('Eveready', 'VERB'), ('Eveready', 'NOUN'))],
 [('what', 'PRON'), (('*T*-252', 'VERB'), ('*T*-252', 'X'))],
 [('ended', 'VERB'), (('up', 'ADV'), ('up', 'PRT'))]]


Adding Tag occurance probability weights: we will apply weights based on the probability of tag occurance to the transition probabilities of tags and then use the resulting probability for predicting unknown words.

This scheme will also take into account that some POS tags are more likely to occur as compared to others.

In [30]:
# lets create a list containing tuples of POS tags and POS tag occurance probability, based on training data
tag_prob = []
total_tag = len([tag for word,tag in train_tagged_words])
for t in T:
    each_tag = [tag for word,tag in train_tagged_words if tag==t]
    tag_prob.append((t,len(each_tag)/total_tag))

tag_prob

[('ADP', 0.0978889970381069),
 ('PRT', 0.03176447193527793),
 ('PRON', 0.0273373313657153),
 ('NOUN', 0.2862674913916711),
 ('VERB', 0.1351167488251855),
 ('DET', 0.08666938784053921),
 ('ADJ', 0.06351847781719991),
 ('X', 0.06576867928872701),
 ('CONJ', 0.02251248076862696),
 ('NUM', 0.035145007169246546),
 ('.', 0.11641391147812072),
 ('ADV', 0.031597015081582885)]

In [31]:
def Viterbi_1(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        p_transition =[] # list for storing transition probabilities
       
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
            # find POS tag occurance probability
            tag_p = [pair[1] for pair in tag_prob if pair[0]==tag ]
            
            # calculate the transition prob weighted by tag occurance probability.
            transition_p = tag_p[0]*transition_p             
            p_transition.append(transition_p)
            
        pmax = max(p)
        state_max = T[p.index(pmax)] 
        
      
        # if probability is zero (unknown word) then use weighted transition probability
        if(pmax==0):
            pmax = max(p_transition)
            state_max = T[p_transition.index(pmax)] 
        else:
            state_max = T[p.index(pmax)] 
        
        state.append(state_max)
    return list(zip(words, state))            

In [32]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi_1(test_tagged_words)
end = time.time()
difference = end-start

In [33]:
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Modified Viterbi_1 Accuracy: ',accuracy*100)

Modified Viterbi_1 Accuracy:  95.57522123893806


In [34]:
# let's check the incorrect tags
incorrect_tagged_cases = [[test_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0]!=j[1]]

In [35]:
incorrect_tagged_cases

[[('military', 'ADJ'), (('command', 'VERB'), ('command', 'NOUN'))],
 [('0', 'X'), (('Sandinista', 'VERB'), ('Sandinista', 'NOUN'))],
 [('its', 'PRON'), (('Eveready', 'VERB'), ('Eveready', 'NOUN'))],
 [('what', 'PRON'), (('*T*-252', 'VERB'), ('*T*-252', 'X'))],
 [('ended', 'VERB'), (('up', 'ADV'), ('up', 'PRT'))]]

Following words have been corrected by Veterbi-1 compared to vanila Viterbi 


Contra tagged as NOUN                                  
Honduras tagged as NOUN                                                            
Offensive tagged as NOUN                                                        
Rebel tagged as NOUN                                                       
Forces tagged as NOUN                                                                        
Bucking tagged as VERB                                                                                                          complaining tagged as VERB


### Viterbi Modification-Technique 2( Rule Based Tagger)


####  Second solution for unknown words:

Backoff to rule based tagger in case of unknown words.
we further observe that POS tag 'X' can be easily encapsulated in regex rule, so we extract it only based on ruled based tagged.
Let's define a rule based tagger as below:

In [36]:
# specify patterns for tagging
patterns = [
    (r'.*ing$', 'VERB'),              # gerund
    (r'.*ed$', 'VERB'),               # past tense 
    (r'.*es$', 'VERB'),               # verb    
    (r'.*\'s$', 'NOUN'),              # possessive nouns
    (r'.*s$', 'NOUN'),                # plural nouns
    (r'\*T?\*?-[0-9]+$', 'X'),        # X
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), # cardinal numbers
    (r'.*', 'NOUN')                   # nouns
]

# rule based tagger
rule_based_tagger = nltk.RegexpTagger(patterns)

In [37]:
# Modification in Viterbi Algorithm : Backoff to rule based tagger in case unknown word is encountered.
def Viterbi_2(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        state_max = rule_based_tagger.tag([word])[0][1]       
       
        
        if(pmax==0):
            state_max = rule_based_tagger.tag([word])[0][1] # assign based on rule based tagger
        else:
            if state_max != 'X':
                # getting state for which probability is maximum
                state_max = T[p.index(pmax)]                
            
        
        state.append(state_max)
    return list(zip(words, state))

In [38]:

# tagging the test sentences
start = time.time()
tagged_seq = Viterbi_2(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

Time taken in seconds:  37.9211688041687


In [39]:
 # accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Modified Viterbi_2 Accuracy: ',accuracy*100)

Modified Viterbi_2 Accuracy:  97.34513274336283


In [40]:
# let's check the incorrect tags
incorrect_tagged_cases = [[test_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0]!=j[1]]

In [41]:
incorrect_tagged_cases

[[('military', 'ADJ'), (('command', 'VERB'), ('command', 'NOUN'))],
 [('apparently', 'ADV'), (('drew', 'NOUN'), ('drew', 'VERB'))],
 [('ended', 'VERB'), (('up', 'ADV'), ('up', 'PRT'))]]

Improvement from Viterbi-1

Following tags have been corrected by Viterbi-2                                      
Sandinista tagged as NOUN                                                             
Eveready tagged as NOUN                                                                 
T*-252 tagged as 'X'                                                  

#### Evaluating tagging accuracy

In [42]:
test_tagged_words = [tup[0] for sent in test_set for tup in sent]
test_run_base = [tup for sent in test_set for tup in sent]

In [43]:

# tagging the test sentences
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)



Time taken in seconds:  1506.4281628131866


In [44]:
# Vanila Viterbi Algorithm Accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Vanila Viterbi Algorithm Accuracy: ',accuracy*100)

Vanila Viterbi Algorithm Accuracy:  90.93390524468707


In [45]:

# tagging the test sentences
start = time.time()
tagged_seq = Viterbi_2(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

Time taken in seconds:  1802.7631123065948


In [46]:
# modified Viterbi Algorithm Accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Modified Viterbi Algorithm Accuracy: ',accuracy*100)

Modified Viterbi Algorithm Accuracy:  95.37921622148568


### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

The accuracy of vanilla Viterbi Algorithm: 90.93%

The accuracy of modified Viterbi Algorithm: 95.38%

### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

Contra tagged as NOUN                                  
Honduras tagged as NOUN                                                            
Offensive tagged as NOUN                                                        
Rebel tagged as NOUN                                                       
Forces tagged as NOUN                                                                        
Bucking tagged as VERB                                                                        
complaining tagged as VERB                                                  
Sandinista tagged as NOUN                                        
Eveready tagged as NOUN                                               
T*-252 tagged as 'X'